-sandbox
# Exercise User Defined Functions
##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Instructions

In this exercise, we're doing ETL. Instead of using built-in functions, use UDFs to complete the exercise. 

As a reminder, that file contains data about people, including:


* first, middle and last names
* gender
* birth date
* Social Security number
* salary

But, as is unfortunately common in data we get from this customer, the file contains some duplicate records. Worse:

* In some of the records, the names are mixed case (e.g., "Carol"), while in others, they are uppercase (e.g., "CAROL"). 
* The Social Security numbers aren't consistent, either. Some of them are hyphenated (e.g., "992-83-4829"), while others are missing hyphens ("992834829").

The name fields are guaranteed to match, if you disregard character case, and the birth dates will also match. (The salaries will match, as well,
and the Social Security Numbers *would* match, if they were somehow put in the same format).

Your job is to remove the duplicate records. The specific requirements of your job are:

* Remove duplicates. It doesn't matter which record you keep; it only matters that you keep one of them.
* Preserve the data format of the columns. For example, if you write the first name column in all lower-case, you haven't met this requirement.
* Write the result as a Parquet file, as designated by *destFile*.
* The final Parquet "file" must contain 8 part files (8 files ending in ".parquet").

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** The initial dataset contains 103,000 records.<br/>
The de-duplicated result haves 100,000 records.

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Getting Started

Run the following cell to configure our "classroom."

In [0]:
%run "./Includes/Classroom-Setup"

Preparing the learning environment...

Preparing the Scala environment...

Preparing the Scala environment...

Preparing the Python environment...

Preparing the Python environment...

...

Initializing Databricks Academy's services for generating dynamic data...

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from wasbs://training@dbtraineastus.blob.core.windows.net/

Cleaning up the learning environment...no actions taken.

Cleaning up the learning environment...no actions taken.

The following functions were defined for you: 
 waitForMLflow 
 ( 
 
 ) 
 MLflow is an open source platform to manage the ML lifecycle, including experimentation, reproducibility and deployment 
 
 display_run_uri 
 ( 
 experiment_id, run_id 
 ) 
 Experiment and run ids 
 
 plot_confusion_matrix 
 ( 
 y_true, y_pred, classes, title=None, cmap=plt.cm.Blues 
 ) 
 Confusion matrix 
 The following variables were defined for you: 
 username : pinky.gtm@mail.kmutt.ac.th 
 No additional information was provided. 
 
 workingDir : dbfs:/user/pinky.gtm@mail.kmutt.ac.th/deep_learning/2_exercise_user_defined_functions_pil 
 No additional information was provided. 
 
 working_path : /dbfs/user/pinky.gtm@mail.kmutt.ac.th/deep_learning/2_exercise_user_defined_functions_pil 
 This is working directory. 
 
 ml_working_path : /dbfs/ml/pinky.gtm@mail.kmutt.ac.th 
 This is ML working directory. 
 All done!

##![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Hints

* Use the <a href="http://spark.apache.org/docs/latest/api/python/index.html" target="_blank">API docs</a>. Specifically, you might find 
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame" target="_blank">DataFrame</a> and
  <a href="http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions" target="_blank">functions</a> to be helpful.
* It's helpful to look at the file first, so you can check the format. `dbutils.fs.head()` (or just `%fs head`) is a big help here.

In [0]:
# TODO

sourceFile = "dbfs:/mnt/training/dataframes/people-with-dups.txt"
destFile = userhome + "/people.parquet"

# In case it already exists
dbutils.fs.rm(destFile, True)

Out[46]: True

In [0]:
df = (
  spark
  .read
  .option('header', True)
  .option('inferSchema', True)
  .option('sep', ':')
  .csv(sourceFile)
)
display(df.limit(5))

firstName,middleName,lastName,gender,birthDate,salary,ssn
Emanuel,Wallace,Panton,M,1988-03-04,101255,935-90-7627
Eloisa,Rubye,Cayouette,F,2000-06-20,204031,935-89-9009
Cathi,Svetlana,Prins,F,2012-12-22,35895,959-30-7957
Mitchel,Andres,Mozdzierz,M,1966-05-06,55108,989-27-8093
Angla,Melba,Hartzheim,F,1938-07-26,13199,935-27-4276


In [0]:
def capitalize(text=None):
  if not text:
    return text
  return text.capitalize()

cap = udf(capitalize)

In [0]:
def normalize_ssn(ssn=None):
  if not ssn:
    return ssn
  temp = ssn.replace('-','')
  return temp[0:3] + '-' + temp[3:5] + '-' + temp[5:]

ssn_norm = udf(normalize_ssn)

In [0]:
from pyspark.sql.functions import col

cleanedDf = (
  df
  .select(
    cap(col("firstName")), 
    cap(col("middleName")), 
    cap(col("lastName")), 
    cap(col("gender")),
    "birthDate",
    "salary",
    ssn_norm(col("ssn")),
  )
  .withColumnRenamed('capitalize(firstName)', 'firstName')
  .withColumnRenamed('capitalize(middleName)', 'middleName')
  .withColumnRenamed('capitalize(lastName)', 'lastName')
  .withColumnRenamed('capitalize(gender)', 'gender')
  .withColumnRenamed('normalize_ssn(ssn)', 'ssn')
  .distinct()
  .repartition(8) # to get 8 parquet files
)

In [0]:
(
  cleanedDf.write                  # Our DataFrameWriter
  .option("compression", "snappy") # One of none, snappy, gzip, and lzo
  .mode("overwrite")               # Replace existing files
  .parquet(destFile)               # Write DataFrame to Parquet files
)

##![Spark Logo Tiny](https://s3-us-west-2.amazonaws.com/curriculum-release/images/105/logo_spark_tiny.png) Validate Your Answer

At the bare minimum, we can verify that you wrote the parquet file out to **destFile** and that you have the right number of records.

Running the following cell to confirm your result:

In [0]:
partFiles = len(list(filter(lambda f: f.path.endswith(".parquet"), dbutils.fs.ls(destFile))))

finalDF = spark.read.parquet(destFile)
finalCount = finalDF.count()

clearYourResults()
validateYourAnswer("01 Parquet File Exists", 1276280174, partFiles)
validateYourAnswer("02 Expected 100000 Records", 972882115, finalCount)
summarizeYourResults()

01 Parquet File Exists:,passed,8
02 Expected 100000 Records:,passed,100000
